In [1]:
import six
import numpy as np

from kids_ggl_pipeline.halomodel import nfw, nfw_stack, satellites, halo, halo_2, halo_2_mc, halo_sz

In [2]:
file = '/Users/cristobal/Documents/kids/KiDS-ACTPol-SZ/kids_ggl_config/new_format/model_sz.config'

In [3]:
class PriorFunction:

    def __init__(self, name):
        self.name = name

    def calculate_prior(self, value, parameters):
        # maybe each prior function should be a separate class
        if self.name == 'uniform':
            return -np.log(parameters[1]-parameters[0]) \
                if parameters[0] < value < parameters[1] else -np.inf
        if self.name == 'normal':
            return -(value-parameters[0])**2 / (2*parameters[1]**2) \
                - np.log(2*np.pi*parameters[1]**2)/2 \
                if parameters[2] < value < parameters[3] else - np.inf
        if self.name == 'lognormal':
            return -(np.log10(value)-parameters[0])**2 / (2*parameters[1]**2) \
                - np.log(2*np.pi*parameters[1]**2)/2 \
                if parameters[2] < value < parameters[3] else - np.inf

# class Parameter(PriorFunction):

#     def __init__(self, values):
        

Perhaps these should be the config `helpers`

In [59]:
class confighod:

    def __init__(self):
        return

    @property
    def valid_priors(self):
        return ('fixed', 'lognormal', 'normal', 'uniform')

    def append_entries(self, section, parameters, starting, line):
        """
        `line` should be a `configline` object
        """
        line = line.split_words()
        if section == 'hod/observables':
            parameters[0] = self.observables(parameters, line)
        elif section == 'hod/ingredients':
            parameters[0].append(self.ingredients(parameters, line))
        # all other cosmo,hod sections
        else:
            if line[0] == 'name':
                if 'mor' in section:
                    parameters[0].append(getattr(relations, line[1]))
                elif 'scatter' in section:
                    parameters[0].append(getattr(scatter, line[1]))
                parameters[1].append([0])
                parameters[2].append(-np.inf)
                parameters[3].append(np.inf)
            else:
                parameters, starting = \
                    self.append_hod_parameters(parameters, starting, line)
        return parameters, starting

    def append_hod_parameters(self, parameters, starting, line):
        """
        To deal with parameters with priors (including fixed values)

        `line` should be a `configline` object
        
        need to consider `join` instances here.
        """
        line, join = line.read_join()
        assert line[1] in self.valid_priors, \
            'prior must be one of {0}'.format(self.valid_priors)
        priors.append(line[1])
        parameters[0].append(float(line[2]))
        # keep adapting the code above to here
        if prior in ('lognormal', 'normal', 'uniform'):
            parameters[1].append(float(line[3]))
            if len(line) > 5:
                parameters[2].append(float(line[4]))
                parameters[3].append(float(line[5]))
            # starting only applies to free parameters
            starting = self.append_starting(starting, parameters, line)
        if len(parameters[2]) < len(parameters[1]):
                parameters[2].append(-np.inf)
                parameters[3].append(np.inf)
            # what's next?
        return parameters, starting
                
    def append_starting(self, starting, parameters, line):
        line, join = configline.read_join(line)
        prior = line[1]
        if prior == 'fixed':
            return starting
        if prior == 'uniform':
            if len(line) == 5:
                starting.append(float(line[4]))
            else:
                starting.append(np.random.uniform(
                    parameters[0], parameters[1], 1)[0])
        elif prior in ('lognormal', 'normal'):
            if len(line) in (5,7):
                starting.append(line[-1])
            elif prior == 'normal':
                starting.append(np.random.normal(
                    parameters[0], parameters[1], 1)[0])
            else:
                starting.append(10**np.random.normal(
                    np.log10(parameters[0]), np.log10(parameters[1]), 1)[0])
        return starting

    def ingredients(self, parameters, line):
        assert line[1] in ('True', 'False'), \
            'Value {1} for parameter {0} in hod/ingredients not valid.' \
            ' Must be True or False'.format(*(line))
        parameters[0].append(line[1] == 'True')
        return parameters

    def observables(self, parameters, line):
        line = configline.split_words(line)
        binning = np.array(line[1].split(','), dtype=float)
        means = np.array(line[2].split(','), dtype=float)
        parameters[0] = [binning[:-1], binning[1:], means]
        return parameters

    def starting_value(self, line):
        line = configline.split_words(line)
        if line[-1].startswith('join'):
            line = line[:-1]
        prior = line[1]
        v1 = float(line[2])
        v2 = float(line[3])
        assert prior in self.valid_priors
        if prior == 'fixed':
            return v1
        if prior == 'uniform':
            if len(line) == 5:
                return float(line[4])
            return np.random.uniform(v1, v2, 1)[0]
        if prior in ('normal', 'lognormal'):
            if len(line) in (5, 7):
                return float(line[-1])
            if prior == 'normal':
                return np.random.normal(v1, v2, 1)[0]
            return 10**np.random.normal(np.log10(v1), np.log10(v2), 1)[0]
            

class configline(str):

    def __init__(self, line):
        self.line = line
        super().__init__()
        return

    def is_comment(self):
        if self.is_empty():
            return False
        return self.line.lstrip()[0] == '#'

    def is_empty(self):
        return len(self.line.lstrip()) == 0

    def is_section(self):
        return self.line[0] == '['

    def read_join(self):
        line = self.split_words()
        if 'join' not in line[-1]:
            return line, False
        return line[:-1], line[-1].split(':')[1]

    def remove_comments(self):
        if '#' in self.line:
            return self.line[:self.line.index('#')]
        return self.line

    def section_name(self):
        return self.line[1:self.line.index(']')]

    def split_words(self):
        _line = '{0}'.format(self.line)
        if isinstance(_line, six.string_types):
            _line = _line.split()
        return _line

    
class configsection(str):

    def __init__(self, section=''):
        self.section = section
        self._parent = None
        super().__init__()
        return

    @property
    def parent(self):
        return self.section.split('/')[0]

    def append_entry_output(self, output, line):
        """
        `line` should be a `configline` object
        """
        line = line.split_words()
        output[0].append(line[0])
        output[1].append(line[1])
        return output

    def append_entry_setup(self, setup, line):
        """
        `line` should be a `configline` object
        """
        line = line.split_words()
        for dtype in (int, float, str):
            setup[0].append(line[0])
            try:
                setup[1].append(dtype(line[1]))
                break
            except ValueError:
                pass
        return setup

    def append_subsection_parameters(self, current, theta):
        if self.section.count('/') == 0:
            theta.append(current)
        elif self.section.count('/') == 1:
            theta[-1].append(current)
        elif self.section.count('/') == 2:
            theta[-1][-1].append(current)
        return theta

#     def parent(self, section):
#         return section.split('/')[0]

In [60]:
class Config:
    
    def __init__(self, filename):
        self.filename = filename
        self._data = None

    @property
    def data(self):
        if self._data is None:
            with open(self.filename) as file:
                data = file.readlines()
            _data = []
            for line in data:
#                 if configline.is_empty(line) or configline.is_comment(line):
                line = configline(line)
                if line.is_empty() or line.is_comment():
                    continue
                if '#' in line:
                    line = line.remove_comments()
                _data.append(line)
            self._data = _data
        return self._data

    @property
    def valid_modules(self):
        return {'satellites': satellites, 'nfw': nfw, 'nfw_stack': nfw_stack,
                'halo': halo, 'halo_2': halo_2, 'halo_2_mc': halo_2_mc,
                'halo_sz': halo_sz, 'halo_sz_modular': halo_sz_modular}

    def initialize_parameters(self):
        return [[] for i in range(4)]

    def read(self):
        section = configsection()
        priors = []
        theta = []
        # starting values for free parameters
        starting = []
        # I wanted to use dictionaries but
        # the problem is they don't preserve order!
        setup = [[], []]
        output = [[], []]
        path = ''
        hod = confighod()
        for line in self.data:
            line = configline(line)
            words = line.split()
            if line[0] == 'model':
                model = self.read_function(words[1])
                continue
            if configline.is_section(line):
                if section == 'cosmo' or section.startswith('hod'):
                    theta = section.append_subsection_parameters(
                        section, parameters, theta)
                section = configsection(configline.section_name(line))
                parameters = self.initialize_parameters()
                continue
            if section == 'setup':
                setup = section.append_entry_setup(setup, words)
            elif section == 'output':
                output = section.append_entry_output(output, words)
            elif section.parent in ('cosmo', 'hod'):
                parameters, starting = \
                    hod.append_entries(section, parameters, line)
        return theta, priors, starting, setup, output

    def read_function(self, path):
        module, func = path.split('.')
        assert module in self.valid_modules, \
            'Implemented modules are {0}'.format(
                np.sort(list(self.valid_modules.keys())))
        return getattr(self.valid_modules[module], func)


In [61]:
cfg = Config(file)
out = cfg.read()

NameError: name 'append_hod_parameters' is not defined

In [121]:
def append_parameters(theta, current, section):
    n = len(theta)
    nc = len(current)
    print('section =', section)
    print('n =', n, nc)
    print(theta, len(theta), len(current))
    if len(theta) > 0:
        print('cosmo =', theta[0])
    if section.count('/') == 0:
        theta.append(current)
    elif section.count('/') == 1:
        theta[-1].append(current)
    elif section.count('/') == 2:
        theta[-1][-1].append(current)
    return theta


def read_config(file):
    section = None
    priors = []
    theta = []
    # starting values for free parameters
    starting = []
    # the problem with dictionaries is they don't preserve order!
    setup = [[], []]
    output = [[], []]
    path = ''
    valid_priors = ('lognormal', 'normal', 'uniform')
    with open(file) as f:
        for line in f:
            line = line.lstrip()
            if len(line) == 0:
                continue
            if line[0] == '#':
                continue
            line = line.split()
            if len(line) == 0:
                continue
            print('line:', line)
            # remove comments at the end
            try:
                line = line[:line.index('#')]
            except ValueError:
                pass

            if line[0] == 'model':
                model = read_function(*(line[1].split('.')))
                continue

            # the start of a section. Time to store the previous section as well
            if line[0][0] == '[':
                new_section = line[0][1:line[0].index(']')]
                # this has to happen only once we're done adding parameters to this section!
                if section is not None:
                    if section == 'cosmo' or section.startswith('hod'):
                        theta = append_parameters(theta, parameters, section)
                    # debug printing
                    if section == 'cosmo':
                        print('== theta ==', theta)
                    else:
                        print('== hod ==', section)
                        print('\t', theta[0])
                        try:
                            print('\t', theta[1])
                        except IndexError:
                            pass
                # this should be read by the sampling helper
                if new_section == 'sampler':
                    break
                section = new_section
                parent_section = section.split('/')[0]
                parameters = [[] for i in range(4)]
                continue

            if section == 'setup':
                for dtype in (int, float, str):
                    setup[0].append(line[0])
                    try:
                        setup[1].append(dtype(line[1]))
                        break
                    except ValueError:
                        pass
            if section == 'output':
                output[0].append(line[0])
                output[1].append(line[1])

            if parent_section in ('cosmo', 'hod'):
                # these are special
                if section == 'hod/observables':
                    binning = np.array(line[1].split(','), dtype=float)
                    means = np.array(line[2].split(','), dtype=float)
                    parameters[0] = [binning[:-1], binning[1:], means]
                elif section == 'hod/ingredients':
                    assert line[1] in ('True', 'False'), \
                        'Value {1} for parameter {0} in hod/ingredients not valid.' \
                        ' Must be True or False'.format(*(line))
                    parameters[0].append(line[1] == 'True')

                # all other cosmo,hod sections
                else:
                    if line[0] == 'name':
                        if 'mor' in section:
                            parameters[0].append(getattr(relations, line[1]))
                        elif 'scatter' in section:
                            parameters[0].append(getattr(scatter, line[1]))
                    elif line[1] == 'fixed':
                        parameters[0].append(float(line[2]))
                    elif line[1] == 'uniform':
                        parameters[0].append(float(line[2]))
                        parameters[1].append(float(line[3]))
                        starting.append(float(line[4]))
                    elif line[1] in ('normal','lognormal'):
                        for i in range(len(line)-2):
                            parameters[i].append(float(line[i+2]))
                        if len(line) == 4 or (len(line) == 5 and line[-1][:4] == 'join'):
                            parameters[2].append(-np.inf)
                            parameters[3].append(np.inf)
                        starting.append(float(line[2]))
                    if line[0] not in ('name','function') and line[1] in valid_priors:
                        priors.append(line[1])
                    # these are all the variable types that only take one value
                    else:
                        if line[1] == 'function':
                            parameters[0].append(read_function(*(line[2].split('.'))))
                        elif line[1] == 'read':
                            fname = line[2]
                            if path:
                                fname = os.path.join(path, fname)
                            v = np.loadtxt(fname, usecols=(int(line[3]),))
                            if not np.iterable(v):
                                v = np.array([v])
                            parameters[0].append(v)

                for i in range(4):
                    if i > 0 and len(parameters[i]) < len(parameters[0]):
                            parameters[i].append(-np.ones_like(parameters[0][-1], dtype=int))
                    print('v{0} = {1}'.format(i+1, parameters[i]))

    # theta = [cosmo, hod]
    theta[1] = theta[1][4:]
    return model, priors, starting, theta, setup, output

In [122]:
cfg = read_config(file)

line: ['model', 'halo_sz_modular.model', '#', 'Simple', 'power-law']
Successfully imported <function model at 0x10ad4d8c8>
line: ['[cosmo]']
line: ['sigma_8', 'fixed', '0.8159']
v1 = [0.8159]
v2 = [-1]
v3 = [-1]
v4 = [-1]
line: ['H0', 'fixed', '67.74']
v1 = [0.8159, 67.74]
v2 = [-1, -1]
v3 = [-1, -1]
v4 = [-1, -1]
line: ['omegam', 'fixed', '0.3089']
v1 = [0.8159, 67.74, 0.3089]
v2 = [-1, -1, -1]
v3 = [-1, -1, -1]
v4 = [-1, -1, -1]
line: ['omegab_h2', 'fixed', '0.02230']
v1 = [0.8159, 67.74, 0.3089, 0.0223]
v2 = [-1, -1, -1, -1]
v3 = [-1, -1, -1, -1]
v4 = [-1, -1, -1, -1]
line: ['omegav', 'fixed', '0.6911']
v1 = [0.8159, 67.74, 0.3089, 0.0223, 0.6911]
v2 = [-1, -1, -1, -1, -1]
v3 = [-1, -1, -1, -1, -1]
v4 = [-1, -1, -1, -1, -1]
line: ['n', 'fixed', '0.9667']
v1 = [0.8159, 67.74, 0.3089, 0.0223, 0.6911, 0.9667]
v2 = [-1, -1, -1, -1, -1, -1]
v3 = [-1, -1, -1, -1, -1, -1]
v4 = [-1, -1, -1, -1, -1, -1]
line: ['z', 'fixed', '0.1884715408']
v1 = [0.8159, 67.74, 0.3089, 0.0223, 0.6911, 0.9667,

In [117]:
len(cfg)

6

In [118]:
print('model:', cfg[0], '\n')
print('priors:', cfg[1], '\n')
print('starting:', cfg[2], '\n')
print('cosmo:', cfg[3][0], '\n')
print('hod:')
for i in cfg[3][1]:
    print(i)
print()
print('setup:', cfg[4], '\n')
print('output:', cfg[5], '\n')

model: <function model at 0x10ad4d8c8> 

priors: ['uniform', 'normal', 'uniform', 'uniform', 'uniform'] 

starting: [1.0, 1.0, 1.0, 0.3, 0.7] 

cosmo: [[0.8159, 67.74, 0.3089, 0.0223, 0.6911, 0.9667, 0.1884715408], [-1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1]] 

hod:
[[array([10.]), array([12.5]), array([11.315])], [array([-1])], [array([-1])], [array([-1])]]
[[True, False, False], [-1, -1, -1], [-1, -1, -1], [-1, -1, -1]]
[[0.0, 1.0], [5.0, -1], [-1, -1], [-1, -1], [[<function powerlaw at 0x10acb9c80>, 14.0, 0.0, 1.0], [-1, -1, -1, 0.5], [-1, -1, -1, 0.0], [-1, -1, -1, 3.0]], [[<function lognormal at 0x10acd90d0>, 0.05], [-1, 3.0], [-1, -1], [-1, -1]], [[0.0, 0.0], [1.0, 3.5], [-1, -1], [-1, -1]]]
[[1.0], [-1], [-1], [-1], [[<function double_powerlaw_scaled at 0x10acd9048>, 0.56], [-1, -1], [-1, -1], [-1, -1]], [[0.0, 0.0, 0.0, 0.0], [-1, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1]]]

setup: [['expansion', 'expansion_stars', 'lnk_bins', 

This is the result I want

In [35]:
hod = [
    [[10],[12.5],[11.315]], # observable
    [True, False], # ingredients
    [ # centrals
        [1, 1], # fc, bias
        ['<powerlaw_function>', 12, 0, 1], # scaling relation
        ['<lognormal_function>', 0.5], # scatter function
        ['<fiducial_miscentring_function>', 0.2, 0.2] # miscentring
    ],
]
hod

[[[10], [12.5], [11.315]],
 [True, False],
 [[1, 1],
  ['<powerlaw_function>', 12, 0, 1],
  ['<lognormal_function>', 0.5],
  ['<fiducial_miscentring_function>', 0.2, 0.2]]]

In [96]:
np.any([[], [], [], []])

False

In [130]:
np.random.uniform(0, 3, 1)[0]

1.3896018862799024